In [2]:
import cv2
from deepface import DeepFace
import time
from collections import defaultdict

2025-01-30 22:08:53.129553: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-30 22:08:53.173328: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-30 22:08:53.179875: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-30 22:08:54.080836: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
cap = cv2.VideoCapture('/dev/video0')

ret, frame = cap.read()
ret


True

In [9]:
def timedCameraRecognition(duration, showRectangle=False):
    
    
    # Dictionary that saves all the emotion data
    emotionMap = defaultdict(int)
    
    if showRectangle:
        # Load pre-trained models for face detection
        faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Open webcam
    cap = cv2.VideoCapture('/dev/video0')

    
    # Run the while loop for a finite amount of time
    # Current time - start time = time elapsed
    startTime = time.time()
    while time.time() - startTime < duration:
        # Capture a frame from webcam
        ret, frame = cap.read()
        if not ret:
            break

        # Run some preprocessing on the frame image
        # frame = preprocessImage(frame)

        # Run DeepFace emotion analysis on frame
        try:
            emotion = DeepFace.analyze(frame, actions='emotion')
            dominantEmotion = emotion[0]['dominant_emotion']
            emotionMap[dominantEmotion] += 1
        except:
            pass #skip if face not detected

        if showRectangle:
            # Draw rectangle around the face
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = faceCascade.detectMultiScale(gray, 1.1, 4)
            for (x,y,w,h) in faces:
                cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2)
            
            # Add text to webcam video
            font = cv2.FONT_HERSHEY_SIMPLEX
            try:
                cv2.putText(frame, dominantEmotion, (50,50), font, 3, (0,0,255), 2, cv2.LINE_4)
            except:
                pass #skip if face not detected
            

        # Display the resulting frame
        cv2.imshow("Emotion Detection", frame)
        
        # Exit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the capture
    cap.release()
    cv2.destroyAllWindows()

    # Save the most frequent emotion from the dictionary to finalEmotion -> Represents the emotion used for robot movement
    sortedEmotionMap = sorted(emotionMap.items(), reverse=True, key=lambda x:x[1])
    finalEmotion = sortedEmotionMap[0][0]
    print(emotionMap)
    print("Most Frequent Emotion: " + finalEmotion)
    return finalEmotion

In [4]:
cap = cv2.VideoCapture("/dev/video0")

if not cap.isOpened():
    print("error")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("error: failed to capture")
        break

    cv2.imshow('cam feed', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cap.destroyAllWindows()

NameError: name 'cap' is not defined

In [13]:
timedCameraRecognition(10, True)

Most Frequent Emotion: sad


'sad'